In [1]:
import requests
import json
from IPython.display import JSON
from keys import *
import pandas as pd
from datetime import datetime

In [2]:
#Be careful not to run too often as there is a maximum of 200 api calls per month

#Barcelona airport code LEBL
url = "https://aerodatabox.p.rapidapi.com/flights/airports/icao/LEBL/2025-09-01T08:00/2025-09-01T20:00"

querystring = {"withLeg": "true", "direction": "Arrival", "withCancelled": "true", "withCodeshared": "true", "withCargo": "true", "withPrivate": "true", "withLocation": "false"}

headers = {
    'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
    'x-rapidapi-key': flights_key
    }

response = requests.request("GET", url, headers=headers, params=querystring)

In [3]:
# Comprueba si la solicitud a la API fue exitosa (código de estado 200)
if response.status_code == 200:
    flight_arrivals = response.json()
    
    data_list = []
    for flight in flight_arrivals['arrivals']:
        data_list.append({
            'scheduled_arrival_time': flight['arrival']['scheduledTime']['local'],
            'flight_number': flight['number'],
            'from_airport': flight['departure']['airport']['name'],
            'airline': flight['airline']['name'],
            'aircraft': flight['aircraft']['model'],
            'icao_code': 'LEBL'
        })
    
    df = pd.DataFrame(data_list)
    df['scheduled_arrival_time'] = pd.to_datetime(df['scheduled_arrival_time']).dt.tz_localize(None)
    
    print(df.head())
    print(f"\nTotal flights: {len(df)}")
    print(f"\nColumn types:\n{df.dtypes}")
    print(f"\nSample datetime value: {df['scheduled_arrival_time'].iloc[0]}")
    print(f"Type: {type(df['scheduled_arrival_time'].iloc[0])}")
else:
    print("Failed to retrieve data from the API. Status code:", response.status_code)

  scheduled_arrival_time flight_number       from_airport  airline  \
0    2025-09-01 08:20:00       IB 5127  Palma De Mallorca   Iberia   
1    2025-09-01 08:20:00       LA 5876  Palma De Mallorca    LATAM   
2    2025-09-01 08:20:00       VY 3907  Palma De Mallorca  Vueling   
3    2025-09-01 08:15:00       V7 2344     Brest/Guipavas  Volotea   
4    2025-09-01 08:15:00       VY 6109               Rome  Vueling   

          aircraft icao_code  
0      Airbus A320      LEBL  
1      Airbus A320      LEBL  
2      Airbus A320      LEBL  
3      Airbus A320      LEBL  
4  Airbus A320 NEO      LEBL  

Total flights: 823

Column types:
scheduled_arrival_time    datetime64[ns]
flight_number                     object
from_airport                      object
airline                           object
aircraft                          object
icao_code                         object
dtype: object

Sample datetime value: 2025-09-01 08:20:00
Type: <class 'pandas._libs.tslibs.timestamps.Timestamp

In [4]:
import sqlalchemy
import pymysql

schema = "gans"
host = "127.0.0.1"
user = "root"
password = "gomaYjeff2705-"
port = 3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [6]:
# Send the flight data to the database
rows_inserted = df.to_sql('flight_arrival', if_exists='append', con=con, index=False)
print(f"Rows inserted: {rows_inserted}")

Rows inserted: 823
